## Description

This Jupyter Notebook is designed to process and extract key details from invoices stored as PDFs. It uses IBM watsonx's granite-3-8b-instruct LLM to interpret the invoice data and extract specific fields such as invoice number, total amounts, and customer details. Extracted data is validated and saved into a structured DataFrame for further analysis.

## Step 1: Setup and Installation

We start by installing the required dependencies. This includes libraries for document conversion, IBM watsonx LLM integration, and data handling.

In [ ]:
!pip install -q git+https://github.com/ibm-granite-community/utils \
    docling==1.13.0 \
    langchain==0.2.12 \
    langchain-ibm==0.1.11 \
    langchain-community==0.2.11 \
    langchain-core==0.2.28 \
    ibm-watsonx-ai==1.1.2 \
    sentence-transformers==3.0.0


## Step 2. Import Libraries and Load Environment Variables

Import necessary libraries and load API credentials and configurations using the dotenv library.

In [7]:
from docling.document_converter import DocumentConverter
from docling.datamodel.base_models import PipelineOptions
from langchain_ibm import WatsonxLLM
from langchain_core.prompts import PromptTemplate
import os
import json
import pandas as pd
import re
from dotenv import load_dotenv, dotenv_values 
load_dotenv() 

True

## Step 3: Define the InvoiceProcessor Class

The InvoiceProcessor class handles the processing of invoices, including document conversion, text extraction using IBM Docling and processing using Granite LLM

In [3]:
# Define the InvoiceProcessor class
class InvoiceProcessor:
    def __init__(self, ibm_cloud_api_key, project_id, watson_url):
        self.llm = WatsonxLLM(
            model_id='ibm/granite-3-8b-instruct',
            apikey=ibm_cloud_api_key,
            project_id=project_id,
            params={
                "decoding_method": "greedy",
                "max_new_tokens": 8000,
                "min_new_tokens": 1,
                "repetition_penalty": 1.01
            },
            url=watson_url
        )

        self.converter = DocumentConverter(
            pipeline_options=PipelineOptions(
                do_table_structure=True,
                do_ocr=True,
                do_cell_matching=False
            )
        )

    def extract_invoice_data(self, source):
        result = self.converter.convert_single(source)
        markdown_output = result.render_as_markdown()
        #print(markdown_output)

        #prompt_template = PromptTemplate.from_template('''
        prompt_template = PromptTemplate(
            input_variables=["DOCUMENT"],
            template='''
            <|start_of_role|>System<|end_of_role|> You are an AI assistant for processing invoices. Based on the provided invoice data, extract the 'Invoice Number', 'Total Net Amount', 'Total VAT or TAX or GST Amount', 'Total Amount' , 'Invoice Date', 'Purchase Order Number' and 'Customer number', without the currency values.

            |Instructions|
            Identify and extract the following information:
            - **Invoice Number**: The unique identifier for the invoice.
            - **Net Amount**: The Total Net Amount indicated on the invoice.
            - **VAT or TAX or GST Amount**: The Total VAT or TAX or GST Amount indicated on the invoice.
            - **Total Amount**: The Total Cost indicated on the invoice.
            - **Invoice Date**: The date the invoice was issued.
            - **Purchase Order Number**: The unique identifier for the purchase order.
            - **Customer Number**: The unique identifier for the customer.

            Invoice Data:
            {DOCUMENT}


            Strictly provide the extracted information in the following JSON format:

            ```json
            {{
              "invoice_number": "extracted_invoice_number",
              "net_amount": "extracted_new_amount",
              "vat_or_tax_or_gst_amount" : "extracted_vat_or_tax_or_gst_amount",
              "total_amount": "extracted_total_amount",
              "invoice_date": "extracted_invoice_date",
              "purchase_order_number": "extracted_purchase_order_number",
              "customer_number": "extracted_customer_number"
            }}

            <|end_of_text|>

            <|start_of_role|>assistant<|end_of_role|>
        ''')

        prompt = prompt_template.format(DOCUMENT=str(markdown_output).strip())
        answer = self.llm.invoke(prompt)
        print(answer)

        json_string = re.search(r'\{.*\}', answer, re.DOTALL).group(0).replace('\n', '')
        data = json.loads(json_string)

        try:
            net_amount = round(float(data['net_amount'].replace(",", "").replace("$", "").strip()), 2)
            vat_or_tax_or_gst_amount = round(float(data['vat_or_tax_or_gst_amount'].replace(",", "").replace("$", "").strip()), 2)
            total_amount = round(float(data['total_amount'].replace(",", "").replace("$", "").strip()), 2)

            data['Validation'] = 'correct' if round(net_amount + vat_or_tax_or_gst_amount, 2) == total_amount else 'check'
        except (ValueError, KeyError):
            data['Validation'] = 'check'

        return data

    def process_invoices(self, folder_path):
        columns = ['File_Name', 'Invoice_Number', 'Net_Amount', 'TAX_Amount', 'Total_Amount', 'Validation', 'Invoice_Date', 'Purchase_Order_Number', 'Customer_Number']
        df_invoice = pd.DataFrame(columns=columns)

        for filename in os.listdir(folder_path):
            if filename.endswith('.pdf'):
                pdf_path = os.path.join(folder_path, filename)
                try:
                    data = self.extract_invoice_data(pdf_path)
                    data['FileName'] = filename

                    new_row = {
                        'File_Name': data['FileName'],
                        'Invoice_Number': data['invoice_number'],
                        'Net_Amount': data['net_amount'],
                        'TAX_Amount': data['vat_or_tax_or_gst_amount'],
                        'Total_Amount': data['total_amount'],
                        'Validation': data['Validation'],
                        'Invoice_Date': data['invoice_date'],
                        'Purchase_Order_Number': data['purchase_order_number'],
                        'Customer_Number': data['customer_number']
                    }

                    df_invoice = pd.concat([df_invoice, pd.DataFrame([new_row])], ignore_index=True)
                except Exception:
                    pass

        return df_invoice


## Step 4. Initialize and Process Invoices

Set up the necessary credentials and folder path to process invoices using the InvoiceProcessor class.

In [6]:
# Main script to initialize and process invoices
ibm_cloud_api_key = os.getenv('ibm_cloud_api_key')
project_id = os.getenv('project_id')
watson_url = os.getenv('watson_url')
folder_path = os.getenv('folder_path')

invoice_processor = InvoiceProcessor(ibm_cloud_api_key, project_id, watson_url)
df_invoice = invoice_processor.process_invoices(folder_path)
df_invoice


KeyboardInterrupt: 